In [1]:
import os
import pandas as pd
import numpy as np

# Configuration settings
from chasm.config import CONFIG

# PLINK preprocessing utilities
from chasm.plink_preprocessing import (
    test, 
    make_AFs, 
    concat_AFs, 
    divide_into_chunks, 
    make_ids,
    is_snp
)